<a href="https://colab.research.google.com/github/jasmis1229/task3-1/blob/main/task3_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
tokenized_corpus_sports = [
    ["축구", "경기", "선수", "패스", "골", "슛"],
    ["야구", "타자", "홈런", "포수", "투수", "스트라이크"],
    ["농구", "리바운드", "슛", "패스", "득점", "경기"],
    ["축구", "골", "선수", "슈팅", "오프사이드", "경기"],
    ["야구", "경기", "타자", "스트라이크", "아웃", "투수"],
    ["농구", "슛", "슛", "3점슛", "득점", "리바운드"]
]

all_words = sorted(set(word for sentence in tokenized_corpus_sports for word in sentence))
word_to_id_sports = {w: i for i, w in enumerate(all_words)}
id_to_word_sports = {i: w for w, i in word_to_id_sports.items()}